In [1]:
!pip install kagglehub
!pip freeze > requirements.txt

import os
import torch
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from pathlib import Path
from collections import Counter
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torch import nn
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from transformers import ViTForImageClassification
from torch.cuda.amp import GradScaler, autocast

Device and AMP setup 

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
scaler = GradScaler() if use_cuda else None
pin_memory = use_cuda

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("iarunava/cell-images-for-detecting-malaria")

print("Path to dataset files:", path)

Path to dataset files: /Users/eliahsand/.cache/kagglehub/datasets/iarunava/cell-images-for-detecting-malaria/versions/1


In [4]:
#!pip install torch torchvision transformers matplotlib




## Preprocessing

In [5]:


# Define the transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to 224x224 for ViT
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard ImageNet normalization
])

# Load the full dataset
dataset = datasets.ImageFolder(path, transform=transform)

# 🔎 Check total class distribution
all_labels = [label for _, label in dataset]
print("📊 Total dataset distribution:", Counter(all_labels))

#Define the split ratios (60% for training, 20% for validation, 20% for testing)
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# 🔎 Check class balance in each split
train_labels = [label for _, label in train_dataset]
val_labels = [label for _, label in val_dataset]
test_labels = [label for _, label in test_dataset]

print("✅ Train:", Counter(train_labels))
print("✅ Val:", Counter(val_labels))
print("✅ Test:", Counter(test_labels))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)


📊 Total dataset distribution: Counter({0: 55116})
✅ Train: Counter({0: 33069})
✅ Val: Counter({0: 11023})
✅ Test: Counter({0: 11024})


## Model, optimizer and loss

In [6]:
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k")
model.classifier = nn.Linear(model.config.hidden_size, 2)
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = CrossEntropyLoss()


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Trainingfunction

In [7]:
def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0.0
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs.logits, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.logits, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    return total_loss / len(loader), correct / total


## Early stopping class

In [8]:
class EarlyStopping:
    def __init__(self, patience=3, delta=0.0):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta

    def __call__(self, val_accuracy, model):
        score = val_accuracy

        if self.best_score is None or score > self.best_score + self.delta:
            self.best_score = score
            self.counter = 0
            torch.save(model.state_dict(), "best_model.pth")
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True


## Model Training

In [9]:
train_losses, val_losses, val_accuracies = [], [], []
early_stopper = EarlyStopping(patience=3)

for epoch in range(20):  # Kan gå opp til 20, men stoppes tidlig
    model.train()
    running_loss = 0.0
    with tqdm(total=len(train_loader), desc=f"Epoch {epoch + 1}", unit='batch') as pbar:
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            if use_cuda:
                with autocast():
                    outputs = model(images)
                    loss = criterion(outputs.logits, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(images)
                loss = criterion(outputs.logits, labels)
                loss.backward()
                optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss / len(train_loader))
            pbar.update(1)

    avg_train_loss = running_loss / len(train_loader)
    val_loss, val_acc = evaluate(model, val_loader, criterion)

    train_losses.append(avg_train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch + 1} - Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%")

    early_stopper(val_acc, model)
    if early_stopper.early_stop:
        print("Early stopping triggered.")
        break


Epoch 1:   1%|          | 3/517 [00:27<1:19:22,  9.27s/batch, loss=0.00313]


KeyboardInterrupt: 

## Plotting results 

In [ ]:
epochs = range(1, len(train_losses) + 1)
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label="Train Loss")
plt.plot(epochs, val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss per Epoch")
plt.grid(True)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, [v*100 for v in val_accuracies], label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.title("Accuracy per Epoch")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

## Testevaluation with best model 

In [ ]:
model.load_state_dict(torch.load("best_model.pth"))
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.logits, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"\nFinal Test Accuracy (best model): {correct / total * 100:.2f}%")